# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

We can understand the type of problem looking at the objective of the supervised learning model.
What we want to predict is whether or not a student needs an early intervention.
So we want to map input features with a function that outputs 0 or 1.
Hence we are classifying the input data to a discrete output. This is a classification problem.

Given the dissociation between the objective (classification) and the estimation criterion (regression), i should clarify the following:
 - We might estimate a regression function, based on the available data, that defines a linear decision boundary that partitions the input space into two class specific regions.
 - Different would be predicting for example a continuous property for a student, which would be a regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [66]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [67]:
# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features
n_features = student_data.shape[1] - 1 # we exclude the target column 'passed'

# Calculate passing students
n_passed = student_data[student_data.passed == "yes"].shape[0]

# Calculate failing students
n_failed = student_data[student_data.passed == "no"].shape[0]

# Calculate graduation rate
grad_rate = (n_passed / float(n_students)) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [68]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [69]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [70]:
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test,
                                                    train_size=num_train, random_state=33)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

The structure of the data is the following:
 - data points are 395
 - features are 31

Considering that the student dataset is small with a large set of features, its not unexpected to get a lot of noise or statistically irrelevant results.
The curse of dimensionality applies to every supervised algorithm. Since every machine learning algorithm is trying to find patterns, they all need to look at the interactions between every possible combination of features. The set of combinations of variables grows exponentially with the number of features. This isn't true of something like linear regression, but it is true of SVMs, Neural Networks, Decision Trees, and anything that can represent arbitrary functions.

In this problem also we have limited resources where space, learning, and query time all matter, not just prevision performance. By looking into the complexity of each of the three model it can serve as a basic heuristic to reduce one or more of these costs as much as possible.

To assess if our prediction is reasonable, besides from comparing it with the median, the mean and checking if it is included in one standard deviation range, we can use SKlearn functions to find the nearest neighbours of the feature vector. We can then contrast your results with the closest neighbours, the ones that have similar characteristics.

Labels also might be unbalanced/sparse.

The three supervised learning model i choose for this problem are the following:
 - SVM
 - KNN
 - Neural Networks ( i'm using scikit-learn master branch - v0.18 rev0.0.4 )

I analyze how each of these interact with the dataset when it comes to training and predicting:

=== SVM ==

the general applications for this model are:
  - classification of images
  - Hand-written characters can be recognition.
  - biological and other sciences. They have been used to classify proteins with up to 90% of the compounds classified correctly

Support vector machines use optimal margin classifier to create margins that separate data.

Rather than linear classifiers, that use the vector θ to desribe the parameters for the margins, SVM use parameters vectors w, b, and write the classifier as h(x) = g(w' * x + b). The decision boundary (the line separating the positive and negative examples) corresponding to is given by: (w, b). The vector w is orthogonal (at 90◦) to the separating hyperplane.
Solving a constrained optimization problem it is possible the obtain the parameters that maximizes the margin. SVM use a quadratic optimization problem to avoid ‘local minimum’ issues of neural nets for example. The solutions define "support vectors" can be used, with a product of the input data, to make predictions. Support vectors lie on the few data points closest to the decision boundary (or hyperplane).

The notion of kernels then give a way to apply SVMs efficiently in very high dimensional feature spaces. “kernel functions” offer a new efficient separability of non-linear regions. They are a generalization of ‘similarity’ to new kinds of similarity measures based on dot products.
”kernel function” defines inner products in the transformed space. Or, it defines similarity in the transformed space. The idea is to gain linearly separation by mapping the data to a higher dimensional space. A set, for example, might not be separated by a linear function, but can be separated by a quadratic one.

With SVM there can be lots of input features. Feature scaling is very important for SVM when we use non linear kernels, expecially before using the Gaussian Kernel. Scaling all of the features such that they have mean zero and unit variance greatly helps with the performance of SVM, since each feature can be compared equivalently to the others, giving better results in dot products.

We also have to choose the 'C' parameter for regularization for obtaining the decision boundary. When we wish to regularize more (reduce overfitting), we decrease C, and when we wish to regularize less (reduce underfitting), we increase C.
This large margin is only achieved when C is very large. Data is linearly separable when a straight line can separate the positive and negative examples. If we have outlier examples that we don't want to affect the decision boundary, then we can reduce C. We could also train C and the parameters for the kernel function using the training and cross-validation datasets.
 
=== KNN ==

The nearest neighbor search problem arises in numerous fields of application, including:
 - Pattern recognition - in particular for optical character recognition
 - Computer vision
 - Computational geometry - see Closest pair of points problem
 - Robotic sensing
 - Internet marketing - see contextual advertising and behavioral targeting
 - Chemical similarity

K nearest neighbors algorithm is a lazy learner. In contrast to eager learners, lazy learners do not compute a function to fit the training data before new data is received.

KNN, in one dimension, produce for each training point a horizontal segment that extends from half the distance to the point on the left to half the distance of the point on the right.
More generally, we can use an arbitrary distance function to define a nearest neighbor.
One way to reduce the variance is local averaging: instead of just one neighbor, we find K and average their predictions.
New instances are compared to the training data itself to make a classification or  regression judgment. Lazy learners have advantages in local scale estimation and easy integration of additional training data.

 - The training time for KNN is fast compared with linear regresion with has to obtain coefficients.
 - KNN have higher memory requirements since it stores all the training data, and linear regression only stores function coefficients.
 - Prediction time is slower in KNN than linear regression because new instances must be fit to the whole data set.

In our problem we do not need a limited model space since we do not have a large dataset, so I would probably point to using KNN as opposed to linear regression.
Moreover if the data cannot be represented by a linear regression (which assumes each feature is independent) it is perhaps better to use KNN which might be better to obtain a higher F1 score on the test set.

=== NN ===

the general applications for this model are:
  - Classification, including pattern and sequence recognition, novelty detection and sequential decision making
  - Robotics, including directing manipulators, prosthesis.
  - Control, including Computer numerical control

Neural Network model is different from logistic regression, in that between the input and the output layer, there can be one or more hidden layers layers.
The leftmost layer, known as the input layer, consists of a set of neurons representing the input features. Each neuron in the hidden layer transforms the values from the previous layer with a weighted linear summation followed by a non-linear activation function like the hyperbolic tan function. The output layer receives the values from the last hidden layer and transforms them into output values.

The advantages of Multi-layer Neural Networks are:
 - Capability to learn non-linear models.
 - Capability to learn models in real-time (on-line learning) using partial_fit.

The disadvantages of Multi-layer Neural Networks include:
 - The training time is high compared to SVM and KNN
 - NN with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.
 - NN requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.
 - NN is sensitive to feature scaling.

NN classifier also uses parameter alpha for regularization (L2 regularization) term which helps in avoiding overfitting by penalizing weights with large magnitudes.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [71]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [98]:
# Import the three supervised learning models from sklearn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Initialize the three models
clf_A = svm.SVC(kernel='linear', random_state=1)
clf_B = KNeighborsClassifier()
clf_C = MLPClassifier(algorithm='l-bfgs', alpha=1e-5, hidden_layer_sizes=(60,), random_state=1)

# Set up the training set sizes
for size in [100, 200, 300]:
# Execute the 'train_predict' function for each classifier and each training set size
    train_predict(clf_A, X_train[:size], y_train[:size], X_test, y_test)
    
for size in [100, 200, 300]:
    train_predict(clf_B, X_train[:size], y_train[:size], X_test, y_test)
    
for size in [100, 200, 300]:
    train_predict(clf_C, X_train[:size], y_train[:size], X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0053 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.9804.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.6207.
Training a SVC using a training set size of 200. . .
Trained model in 0.0230 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 0.8963.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.7328.
Training a SVC using a training set size of 300. . .
Trained model in 0.1041 seconds
Made predictions in 0.0027 seconds.
F1 score for training set: 0.8678.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7368.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0005 seconds
Made predictions in 0.0017 seconds.
F1 score for training set: 0.8957.
Made predictions in 0.0016 seconds.
F1 score for test set: 0.7259.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.00

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVC

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0063                  | 0.0007                 | 0.9804           | 0.6207          |
| 200               | 0.0356                  | 0.0018                 | 0.8963           | 0.7328          |
| 300               | 0.1073                  | 0.0027                 | 0.8678           | 0.7368          |

** Classifer 2 - KNeighborsClassifier 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0004                  | 0.0011                 | 0.8957           | 0.7259          |
| 200               | 0.0005                  | 0.0029                 | 0.8754           | 0.7368          |
| 300               | 0.0006                  | 0.0059                 | 0.8764           | 0.7442          |

** Classifer 3 - MLPClassifier

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0681                  |  0.0007                | 1.0000           | 0.6667          |
| 200               | 0.2903                  |  0.0009                | 1.0000           | 0.7107          |
| 300               | 0.3954                  |  0.0012                | 0.9976           | 0.7227          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

SVM is my choice.

 - Because it cost money to run these algorithms on a third party vendor's computers, we need to consider not just the F1 score performance, but the computation resources each algorithm needs to use. Since the board of supervisors has asked the most effective model that uses the least amount of computation costs to save on the budget, we can use SVM over KNN. KNN also needs to keep in memory all the input data, which might be a problem if students increase a lot. SVM is memory efficient.
 - KNN is twice as expensive as SVM for making predictions on new data. This is an online/realtime service, so we might get many queries per second. The prediction time should be low.
 - I expect we need to train the model again only when the course period is over, so training time can be relatively high.
 
SVM might not perform as good as Neural Network in some cases. In this case NN suffers from overfitting and we would need to tune the neural network iterations and hidden layers.

Model reshaping could improove SVM computational performance. It consists in selecting only a portion of the available features to fit a model.

Scaling needs to be used whenever our classifier considers differences in multiple features simultaneously, like SVM.
It could be the case that this data is already mostly normally distributed and equivalently scaled. In that case, scaling won't make a big difference in performance, but it may be hard to say whether this is the case beforehand.

The kernel function that is actually performing better for this kind of problem is the Linear Function rather than the Radial Basis function.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

SVM model can determine effectively, based on individual features, the student current position.
The SVM performance has shown to increase as the number of classified students increase.
This choise is effective given our student input data.
It is also versatile as we can tune the algorithm parameters for reducing the impact of unbalanced input data.

We describe how an SVM is trained and how the model makes predictions. Thus please discuss the  and the kernel trick and how an SVM actually makes a prediction. As a straight forward way to explain the algorithm is with a concrete example with the students in this dataset. Thus the passing students could be blue and red the failing students...

Our algorithm distinguish between the 2 types of stundents in input data, "passing" and "not passing".
Our algorithm tries to find a maximum margin that separates the dataset in two groups.
Students have many attributes, so the curve that separated the two groups might not be linear.
It is called hyper-plane since it is n-dimensional curve.
Our algorithm uses a function called "kernel function" to build a new feature, based on the attributes provided for each student, to find an hyperplane that reduce outliers and classifies well the data.
"kernel functions" takes low dimensional input space and transform it to a higher dimensional space. For example it converts not separable problem to separable problem, these functions are called kernels. It is mostly useful in non-linear separation problem, doing data transformations.

Once we have found this hyper-plane, we can analyze new data attributes and make predictions if the student is passing or not.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [104]:
from sklearn import grid_search
from sklearn.metrics import make_scorer
from sklearn import svm

# from sklearn.preprocessing import StandardScaler  
# scaler = StandardScaler()  
## fit only on training data
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
## apply same transformation to test data
# X_test = scaler.transform(X_test)  

# replace yes, no
# y_train = y_train.replace(['yes', 'no'], [1, 0])

# Create the parameters list you wish to tune
parameters = {'kernel':['linear', 'rbf', 'sigmoid'],
              'C':[0.01, 0.1, 1.0, 10.0, 100.0],
              'random_state':[None,42],
              'class_weight':[None,'balanced']}

# Initialize the classifier
clf = svm.SVC(random_state=3)

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(grid_obj, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(grid_obj, X_test, y_test))

Made predictions in 0.0058 seconds.
Tuned model has a training F1 score of 0.8258.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.7248.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

Tuned model has a training F1 score of 0.8258
Tuned model has a testing F1 score of 0.7248
The untuned model has a testing F1 score fo 0.7368

The model has a relatively high score on training data but falls below the score fo the untuned model.
There is no absolute guarantee that grid search will always find the absolute best model. This depends on not only choosing the correct metric and number of parameters in grid space, but if the sample size of the training data is sufficiently large enough to not only to train the model but also big enough to validate on.
In this case grid search will determine the best model from the average score from training the model on two points and using one point as the validation. However as mentioned above if we have very few points to begin with and a lot of features, it wouldn't be statistically significant.

Ordinary single split cross validation (rather than k-fold) doesn't give an accurate measure of the performance of the algorithm for this small a dataset. Indeed we find that what looked like the best model turned out to be the worst when we checked it using GridSearchCV (it was just the most accurate model on that particular data split but performed much worse on other splits).

The grid search algorithm exhaustively considers all parameter combinations of learning algorithm of the "validation set" to be able to tune or optimize the algorithm configuration.
In particular the grid search technique is configured with a grid that specifiess how the search method computes the parameters in the ranges chosen for the model.
The parameter C, common to all SVM kernels, trades off misclassification of training examples against simplicity of the decision surface. A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly. Gamma defines how much influence a single training example has. The larger gamma is, the closer other examples must be to be affected.
Proper choice of C and gamma is critical to the SVM’s performance.

Also by partitioning the available data like before question 2, we reduce the number of samples which can be used for learning the model, and the model prevision score can depend on a particular random choice for the pair of (train, validation) sets. A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV.
In the basic approach, called k-fold CV, the training set is split into k smaller sets. The following procedure is followed for each of the k “folds”:
 -  A model is trained using "k-1" of the folds as training data;
 -  the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).  

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set).

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.